In [4]:
from IPython.core.display import display, HTML,clear_output, Image
import ipywidgets as widgets
import rasterio
import glob
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
def plot_location(df_c,df_o,lat,lon):
    c_df = df_c[df_c.intersects(Point(lon,lat))].copy()
    if len(c_df)!=0:
        c = c_df['CNTRY_NAME'].tolist()[0]
        display(HTML(f'<h3><left>The corrdinates are in {c}</left></h3>'))
        c_df['geometry'].plot(figsize=(5,5))
        plt.show(plt.scatter(lon,lat,color='r',s=200))
    else:
        o_df = df_o[df_o.intersects(Point(lon,lat))].copy()
        if len(o_df)!=0:
            o = o_df['name'].tolist()[0]
            display(HTML(f'<h2 style="background-color:powderblue;"><left>oops.. your coordinates are in the {o}</left></h2>'))
            display(Image(filename="fish.png"))
            return 0
        else:
            display(HTML('<h2 style="background-color:red;"><left>unknown country, your coordinates do not make any sense :)  please check again the coordinates</left></h2>'))
            display(Image(filename="stop_sign.png"))
            return 0


def get_cloud_cover(lat,lon,i,status):
    if status!=0:
        r = rasterio.open(i)
        cc = r.read(1)[r.index(lon,lat)]
#         date = pd.to_datetime(i.split('_')[4]) # this is if the TIFFs are in current folder
        head,tail = os.path.split(i)
        date = pd.to_datetime(tail.split('_')[4])
        if isinstance(date,pd.datetime):
            return {'Date':date,
                    'Cloud Cover': cc}
        else:
            print (f'problem with date: {date}')
    else:
        return 0


def plot_results(df,status):
    if status!=0:
        df.sort_values('Date',inplace=True)
        plt.figure(figsize=(20,5))
        plt.plot(df['Date'],df['Cloud Cover'],'o-')
        plt.ylim(0,100)
        plt.xticks(df['Date'].tolist(),rotation=90, size=17) 
        plt.yticks(size=17)
        plt.ylabel('Monthly Cloud Cover %',size=20)
        plt.axhline(y=60, color='g', linestyle=(0,(5,10)))
        plt.axhline(y=80, color='orange', linestyle=(0,(5,10)))
        plt.axhline(y=90, color='r', linestyle=(0,(5,10)))
        plt.title('monthly average of cloud cover over the coordinates (10X10 km pixel) at 10:30 AM local time', size=15)
        plt.grid();
    else:
        print('no data')

In [2]:
df_countries = gpd.read_file("countries.geojson")
df_oceans = gpd.read_file("Oceans_bounderies.geojson")
rasters_list = glob.glob('./rasters_as_int/*TIFF')

In [ ]:
display(HTML('<h1><left>Global Cloud Tool</left></h1>'))
display(HTML('<h4><left>Check cloud cover per coordinates</left></h4>'))
Image(filename="clouds.png")#,width=200

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
# !jupyter serverextension enable voila --sys-prefix

In [ ]:
name = widgets.Text(placeholder='Please enter coordinates (lat,lon)')

In [ ]:
button_send = widgets.Button(
                description='Submit',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        lat,lon = float(name.value.split(',')[0]),float(name.value.split(',')[1])
        status = plot_location(df_countries,df_oceans,lat,lon)
        plt.show(plot_results(pd.DataFrame([get_cloud_cover(lat,lon,i,status) for i in rasters_list]),status))

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [ ]:
text_1 = widgets.HTML(value="<h3>Please enter coordinates (lat,lon)</h3>")
vbox_text = widgets.VBox([text_1,name,vbox_result])

In [ ]:
page = widgets.HBox([vbox_text])
display(page)